Neighborhood Patterns documentation:

https://docs.safegraph.com/docs/neighborhood-patterns


Social Distancing Metric:

https://docs.safegraph.com/docs/social-distancing-metrics



Using SafeGraph Weekly Patterns Dataset to serve as a proxy for discontinued social distancing metrics
https://colab.research.google.com/drive/1ETZo3KBhcwUikLRos5YmMZkf4aPweG44#scrollTo=11r70CTeAdXi

The contribution of wildfire to PM2.5 trends in the USA
https://www.nature.com/articles/s41586-023-06522-6

Wildfire smoke impacts respiratory health more than fine particles from other sources: observational evidence from Southern California
https://www.nature.com/articles/s41467-021-21708-0

In [10]:
# ! pip install CensusData

In [1]:
import pandas as pd
import os
from tqdm import tqdm
tqdm.pandas()
import geopandas as gpd
import glob
pd.set_option('display.max_columns', None)
import ast
import numpy as np
import json

# Helper functions

In [2]:
def get_all_files(root_dir, contains=[''], extions=['']):
    found_files = []
    for rt_dir, dirs, files in os.walk(root_dir):
        for ext in extions:
            ext = ext.lower()
            ext_len = len(ext)
            for file in files:
                file_ext = file[-(ext_len):]
                # print(file)
                file_ext = file_ext.lower()
                if file_ext == ext:
                    file_name = os.path.join(rt_dir, file)
                    found_files.append(file_name)
                    # continue                    
                
        for con in contains:
            con = con.lower()
            con_len = len(con)
            for file in files:
                if con in os.path.basename(file):
                    file_name = os.path.join(rt_dir, file)
                    found_files.append(file_name)
    return found_files

# Get files

In [3]:
data_root_dir = r'F:\SafeGraph\Advan_2023_API'

all_files = get_all_files(data_root_dir)
print(f"Found files: {len(all_files)}")
print("The top 5 and bottom 5 files:")
all_files[:5] + ['...'] + all_files[-5:]

Found files: 0
The top 5 and bottom 5 files:


['...']

In [37]:
target_years = ['2023']
target_months = ['06'] # ['08', '06', '07']
target_dataset = ['Neighborhood_Patterns']
# target_names = ['neighborhood_patterns_']
target_files = []

for file in all_files[:]:
    directories = file.replace(data_root_dir, '').split(os.sep)[1:]
    # print(directories)
    if len(directories) < 5:
        continue
    year = directories[1]
    month = directories[2]
    dataset = directories[0]
    basename = directories[-1]
    if year in target_years:
        if month in target_months:            
            if dataset in target_dataset:
                # for target_name in target_names:
                    # print("Year, Month, dataset, target_name:", year, month, dataset, target_name)
                    # if target_name in basename:
                target_files.append(file)

print(f"Found target files: {len(target_files)}")
print("The top 5 and bottom 5 files:")

Found target files: 11
The top 5 and bottom 5 files:


In [5]:
# df = pd.read_csv(target_files[0])
df = pd.concat([pd.read_csv(f) for f in target_files[:]])
df

,AREA,AREA_TYPE,ORIGIN_AREA_TYPE,DATE_RANGE_START,DATE_RANGE_END,DAY_COUNTS,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,STOPS_BY_DAY,STOPS_BY_EACH_HOUR,DEVICE_HOME_AREAS,WEEKDAY_DEVICE_HOME_AREAS,WEEKEND_DEVICE_HOME_AREAS,BREAKFAST_DEVICE_HOME_AREAS,LUNCH_DEVICE_HOME_AREAS,DINNER_DEVICE_HOME_AREAS,NIGHTLIFE_DEVICE_HOME_AREAS,WORK_HOURS_DEVICE_HOME_AREAS,WORK_BEHAVIOR_DEVICE_HOME_AREAS,DEVICE_DAYTIME_AREAS,DISTANCE_FROM_HOME,DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION,MEDIAN_DWELL,TOP_SAME_DAY_BRAND,TOP_SAME_MONTH_BRAND,POPULARITY_BY_EACH_HOUR,POPULARITY_BY_HOUR_MONDAY,POPULARITY_BY_HOUR_TUESDAY,POPULARITY_BY_HOUR_WEDNESDAY,POPULARITY_BY_HOUR_THURSDAY,POPULARITY_BY_HOUR_FRIDAY,POPULARITY_BY_HOUR_SATURDAY,POPULARITY_BY_HOUR_SUNDAY,DEVICE_TYPE,ISO_COUNTRY_CODE,REGION,Y,M
0,470370112006,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",6558,4181,"[208,162,223,224,92,215,261,174,260,218,326,15...","[0,0,0,0,0,0,0,0,22,22,39,6,14,26,28,0,7,10,5,...","{""470370112006"":148,""470370153004"":47,""4703701...","{""470370112006"":103,""470370153004"":34,""4703701...","{""470370112006"":15,""470370110022"":4,""470370116...","{""470370112006"":7,""470370110022"":4,""4703701160...","{""470370112006"":15,""470370153004"":4,""470370110...","{""470370112006"":31,""470370153004"":8,""470370110...","{""470370153004"":4,""470370110022"":4,""4703701950...","{""470370112006"":20,""470370153004"":10,""47037011...","{""470370193001"":4,""470370119002"":4,""4703701562...","{""470370112006"":95,""470370195004"":53,""47037019...",3940.0,6356.0,32.066667,"{""Mall"":15,""Dollar Tree"":7,""Kroger"":7,""Walmart...","{""Mall"":64,""Walmart"":40,""Kroger"":39,""Gas Stati...","[9,17,18,21,20,7,7,10,33,44,46,44,50,69,45,9,1...","[137,143,151,147,146,151,142,180,135,189,172,2...","[126,146,150,122,122,151,157,176,245,191,164,2...","[127,141,147,146,152,151,124,175,165,191,211,2...","[126,141,133,139,162,153,142,149,182,242,321,3...","[159,131,131,139,133,148,129,166,161,119,132,1...","[70,83,89,89,77,91,98,116,118,155,93,74,127,17...","[131,127,148,157,156,173,193,214,190,240,300,2...","{""android"":1630,""ios"":1323}",US,TN,2023,6
1,310199694003,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",42090,8019,"[1842,1829,1559,1210,1200,1150,1338,1712,1755,...","[0,10,17,23,51,16,89,56,90,133,116,174,183,106...","{""310199694003"":560,""310199692031"":242,""310199...","{""310199694003"":469,""310199692031"":195,""310199...","{""310199694003"":128,""310199692031"":31,""3101996...","{""310199694003"":104,""310199692031"":27,""3101996...","{""310199694003"":141,""310199692031"":93,""3101996...","{""310199694003"":143,""310199692031"":51,""3101996...","{""310199694003"":6,""310199692031"":4,""3101996920...","{""310199694003"":247,""310199692031"":145,""310199...","{""310199694003"":5,""310199690001"":8,""3101996920...","{""310199694003"":665,""310199692031"":248,""310199...",1988.0,1287.0,10.483333,"{""Hospitals"":22,""Walmart"":21,""Mall"":20,""Gas St...","{""Walmart"":54,""Mall"":53,""Gas Stations"":41,""Hos...","[110,114,124,133,164,167,225,236,230,239,263,3...","[324,345,348,362,356,478,574,739,683,817,824,8...","[334,378,362,398,425,483,582,691,833,869,881,9...","[449,470,423,465,494,593,664,783,797,775,715,8...","[423,477,462,474,503,589,837,964,1068,1007,112...","[485,512,505,516,619,712,896,1085,1245,1110,12...","[391,430,384,427,416,512,487,515,565,624,613,6...","[443,433,415,465,485,512,511,516,594,635,685,6...","{""android"":2740,""ios"":2349}",US,NE,2023,6
2,550939606003,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",33042,4781,"[1594,1195,789,1047,930,927,726,928,1295,1130,...","[0,0,5,42,74,42,23,59,82,145,70,72,81,71,64,11...","{""550939606003"":305,""550939606001"":85,""2704908...","{""5

In [16]:
df.columns

Index(['AREA', 'AREA_TYPE', 'ORIGIN_AREA_TYPE', 'DATE_RANGE_START',
       'DATE_RANGE_END', 'DAY_COUNTS', 'RAW_STOP_COUNTS', 'RAW_DEVICE_COUNTS',
       'STOPS_BY_DAY', 'STOPS_BY_EACH_HOUR', 'DEVICE_HOME_AREAS',
       'WEEKDAY_DEVICE_HOME_AREAS', 'WEEKEND_DEVICE_HOME_AREAS',
       'BREAKFAST_DEVICE_HOME_AREAS', 'LUNCH_DEVICE_HOME_AREAS',
       'DINNER_DEVICE_HOME_AREAS', 'NIGHTLIFE_DEVICE_HOME_AREAS',
       'WORK_HOURS_DEVICE_HOME_AREAS', 'WORK_BEHAVIOR_DEVICE_HOME_AREAS',
       'DEVICE_DAYTIME_AREAS', 'DISTANCE_FROM_HOME',
       'DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION', 'MEDIAN_DWELL',
       'TOP_SAME_DAY_BRAND', 'TOP_SAME_MONTH_BRAND', 'POPULARITY_BY_EACH_HOUR',
       'POPULARITY_BY_HOUR_MONDAY', 'POPULARITY_BY_HOUR_TUESDAY',
       'POPULARITY_BY_HOUR_WEDNESDAY', 'POPULARITY_BY_HOUR_THURSDAY',
       'POPULARITY_BY_HOUR_FRIDAY', 'POPULARITY_BY_HOUR_SATURDAY',
       'POPULARITY_BY_HOUR_SUNDAY', 'DEVICE_TYPE', 'ISO_COUNTRY_CODE',
       'REGION', 'Y', 'M'],
      dtype='obje

In [17]:
df['AREA'] = df['AREA'].astype(str).str.zfill(12)
df['M'] = df['M'].astype(str).str.zfill(2)

In [18]:
CBG_count = len(df['AREA'].unique())
CBG_count

220684

In [19]:
len(str(sorted(df['AREA'].to_list())[0]))

12

## Get the CBG list

In [20]:
# get the CBG list
CBG_file_name = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_list.csv"
# df[['area']].sort_values('area').to_csv(CBG_file_name, index=False)


In [21]:
np.ones((220684, 720)).shape

(220684, 720)

## Create a CBG diction

In [22]:
CBG_dict_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_dict.json"

In [24]:
CBG_df = pd.read_csv(CBG_file_name, dtype={'area':str})
CBG_dict = {row['area']:idx for idx, row in CBG_df.iterrows()}
# json.dump(CBG_dict, open(CBG_dict_file, 'w'))

In [25]:
# CBG_dict
CBG_df

,area
0,010010201001
1,010010201002
2,010010202001
3,010010202002
4,010010203001
...,...
220679,780309611001
220680,780309611002
220681,780309612001
220682,780309612002


In [27]:
def CBG_list_to_Sqlite(df, sqlite_name):
     
    conn = sqlite3.connect(sqlite_name)
    curs = conn.cursor()
    
    columns = ','.join(df.columns)
    columns.replace('placekey,parent_placekey', 'placekey PRIMARY KEY,parent_placekey')
    
    curs.execute('create table if not exists POI ' +
                f"({','.join(df.columns)})")
    df.to_sql('POI', conn, if_exists='replace', index=False)
    
    sql = f'CREATE INDEX placekey_idx ON POI(placekey);'    
    curs.execute(sql)
    
    conn.close

In [14]:
len(ast.literal_eval(df.iloc[0]['stops_by_each_hour']))#[0][0][0]

720

In [15]:
df.columns

Index(['area', 'area_type', 'origin_area_type', 'date_range_start',
       'date_range_end', 'day_counts', 'raw_stop_counts', 'raw_device_counts',
       'stops_by_day', 'stops_by_each_hour', 'device_home_areas',
       'weekday_device_home_areas', 'weekend_device_home_areas',
       'breakfast_device_home_areas', 'lunch_device_home_areas',
       'afternoon_tea_device_home_areas', 'dinner_device_home_areas',
       'nightlife_device_home_areas', 'work_hours_device_home_areas',
       'work_behavior_device_home_areas', 'device_daytime_areas',
       'distance_from_home', 'distance_from_primary_daytime_location',
       'median_dwell', 'top_same_day_brand', 'top_same_month_brand',
       'popularity_by_each_hour', 'popularity_by_hour_monday',
       'popularity_by_hour_tuesday', 'popularity_by_hour_wednesday',
       'popularity_by_hour_thursday', 'popularity_by_hour_friday',
       'popularity_by_hour_saturday', 'popularity_by_hour_sunday',
       'device_type', 'iso_country_code', 're

In [26]:
df.iloc[0]['stops_by_each_hour']

'[0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,20,0,13,0,0,0,0,15,76,27,40,93,12,0,0,0,14,7,6,0,0,0,0,27,12,7,6,0,37,0,24,0,0,0,0,0,0,0,28,15,0,0,14,0,0,0,0,0,18,21,15,53,0,0,0,0,9,8,7,39,34,38,0,0,16,16,7,7,0,16,42,25,9,9,0,11,20,26,20,35,34,46,41,20,71,13,19,23,14,0,9,10,0,13,0,0,13,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,10,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,15,16,13,0,0,0,24,0,0,0,9,15,46,0,0,0,0,0,0,23,40,24,24,19,10,29,0,0,0,22,43,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,0,0,0,0,0,0,0,0,0,0,14,8,63,48,0,38,52,0,0,0,0,0,0,0,0,0,0,0,16,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,61,15,0,0,17,41,28,37,57,22,0,22,0,21,60,33,0,0,0,9,30,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,11,0,0,0,0,0,0,6,0,17,0,38,29,22,27,0,16,13,29,0,7,10,0,34,80,79,36,25,26,0,15,0,16,9,0,0,7,0,0,0,0,0,6,14,0,

In [27]:
df.iloc[0]['popularity_by_each_hour']

'[0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,16,11,16,0,7,0,0,0,0,13,6,12,7,14,15,0,0,0,16,11,12,6,7,11,0,6,9,22,19,0,15,11,12,0,0,0,0,0,0,0,9,16,0,0,7,0,0,0,0,0,23,18,11,33,11,0,0,0,6,7,15,14,12,11,12,0,6,24,10,16,0,12,22,12,11,10,0,15,11,20,22,9,11,25,26,27,22,20,23,13,16,0,6,13,0,10,0,0,6,0,0,0,0,7,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,8,8,12,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,14,7,22,0,0,0,16,13,7,0,14,11,12,12,0,0,0,0,0,7,10,6,13,13,16,16,0,0,0,16,17,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,10,7,6,11,7,16,12,12,0,0,0,0,0,0,0,0,0,0,12,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,17,6,0,0,9,6,12,6,20,17,0,10,0,14,13,9,0,0,0,10,11,9,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,10,7,0,0,0,0,0,15,0,16,10,16,26,27,8,0,16,10,25,15,11,13,8,23,39,15,25,17,26,23,17,22,15,7,0,0,16,0,0,0,0,0,

In [28]:
len(ast.literal_eval(df.iloc[0]['popularity_by_each_hour']))

720

In [29]:
ast.literal_eval(df.iloc[0]['device_home_areas'])
ast.literal_eval(df.iloc[0]['stops_by_day'])
np.array(ast.literal_eval(df.iloc[0]['stops_by_day'])).sum()

4356

In [30]:
np.array(ast.literal_eval(df.iloc[0]['stops_by_each_hour'])).sum()

4367

In [31]:
# 

In [32]:
stops = ast.literal_eval(df.iloc[0]['stops_by_each_hour'])
len(stops)

720

# Create edge list of a hourly matrix

## Load CBGs and their index 

In [34]:
import random
import sqlite3

CBG_file_name = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_list.csv"
# CBG_dict_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_dict.json"
saved_dir = r'D:\SafeGraph\Neighborhood_Patterns_reorganized'

In [35]:
CBG_df = pd.read_csv(CBG_file_name, dtype={'area': str})
CBG_df['CBG_index'] = CBG_df.index
# Pandas dataframe to dictionary. Column "area" as the key, column "CBG_index" as the value.
CBG_dict = CBG_df.set_index('area')['CBG_index'].to_dict()
print("Found CBG count:", len(CBG_dict.keys()))
CBG_df

Found CBG count: 220684


,area,CBG_index
0,010010201001,0
1,010010201002,1
2,010010202001,2
3,010010202002,3
4,010010203001,4
...,...,...
220679,780309611001,220679
220680,780309611002,220680
220681,780309612001,220681
220682,780309612002,220682


In [36]:
n = 5 # for n random key-value pairs
random_keys = random.sample(list(CBG_dict.keys()), n)
random_items = {key: CBG_dict[key] for key in random_keys}

print(random_items)

{'361031587113': 141604, '320030078003': 118456, '420770096003': 170213, '250010130021': 90815, '360010132003': 127774}


In [33]:
np_df = df
np_df['M'] = np_df['M'].astype(str).str.zfill(2)
year = np_df.iloc[0]['Y']
month = np_df.iloc[0]['M']
print(f"Year, month: {year}-{month}")

np_df

Year, month: 2023-06


,AREA,AREA_TYPE,ORIGIN_AREA_TYPE,DATE_RANGE_START,DATE_RANGE_END,DAY_COUNTS,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,STOPS_BY_DAY,STOPS_BY_EACH_HOUR,DEVICE_HOME_AREAS,WEEKDAY_DEVICE_HOME_AREAS,WEEKEND_DEVICE_HOME_AREAS,BREAKFAST_DEVICE_HOME_AREAS,LUNCH_DEVICE_HOME_AREAS,DINNER_DEVICE_HOME_AREAS,NIGHTLIFE_DEVICE_HOME_AREAS,WORK_HOURS_DEVICE_HOME_AREAS,WORK_BEHAVIOR_DEVICE_HOME_AREAS,DEVICE_DAYTIME_AREAS,DISTANCE_FROM_HOME,DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION,MEDIAN_DWELL,TOP_SAME_DAY_BRAND,TOP_SAME_MONTH_BRAND,POPULARITY_BY_EACH_HOUR,POPULARITY_BY_HOUR_MONDAY,POPULARITY_BY_HOUR_TUESDAY,POPULARITY_BY_HOUR_WEDNESDAY,POPULARITY_BY_HOUR_THURSDAY,POPULARITY_BY_HOUR_FRIDAY,POPULARITY_BY_HOUR_SATURDAY,POPULARITY_BY_HOUR_SUNDAY,DEVICE_TYPE,ISO_COUNTRY_CODE,REGION,Y,M
0,470370112006,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",6558,4181,"[208,162,223,224,92,215,261,174,260,218,326,15...","[0,0,0,0,0,0,0,0,22,22,39,6,14,26,28,0,7,10,5,...","{""470370112006"":148,""470370153004"":47,""4703701...","{""470370112006"":103,""470370153004"":34,""4703701...","{""470370112006"":15,""470370110022"":4,""470370116...","{""470370112006"":7,""470370110022"":4,""4703701160...","{""470370112006"":15,""470370153004"":4,""470370110...","{""470370112006"":31,""470370153004"":8,""470370110...","{""470370153004"":4,""470370110022"":4,""4703701950...","{""470370112006"":20,""470370153004"":10,""47037011...","{""470370193001"":4,""470370119002"":4,""4703701562...","{""470370112006"":95,""470370195004"":53,""47037019...",3940.0,6356.0,32.066667,"{""Mall"":15,""Dollar Tree"":7,""Kroger"":7,""Walmart...","{""Mall"":64,""Walmart"":40,""Kroger"":39,""Gas Stati...","[9,17,18,21,20,7,7,10,33,44,46,44,50,69,45,9,1...","[137,143,151,147,146,151,142,180,135,189,172,2...","[126,146,150,122,122,151,157,176,245,191,164,2...","[127,141,147,146,152,151,124,175,165,191,211,2...","[126,141,133,139,162,153,142,149,182,242,321,3...","[159,131,131,139,133,148,129,166,161,119,132,1...","[70,83,89,89,77,91,98,116,118,155,93,74,127,17...","[131,127,148,157,156,173,193,214,190,240,300,2...","{""android"":1630,""ios"":1323}",US,TN,2023,06
1,310199694003,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",42090,8019,"[1842,1829,1559,1210,1200,1150,1338,1712,1755,...","[0,10,17,23,51,16,89,56,90,133,116,174,183,106...","{""310199694003"":560,""310199692031"":242,""310199...","{""310199694003"":469,""310199692031"":195,""310199...","{""310199694003"":128,""310199692031"":31,""3101996...","{""310199694003"":104,""310199692031"":27,""3101996...","{""310199694003"":141,""310199692031"":93,""3101996...","{""310199694003"":143,""310199692031"":51,""3101996...","{""310199694003"":6,""310199692031"":4,""3101996920...","{""310199694003"":247,""310199692031"":145,""310199...","{""310199694003"":5,""310199690001"":8,""3101996920...","{""310199694003"":665,""310199692031"":248,""310199...",1988.0,1287.0,10.483333,"{""Hospitals"":22,""Walmart"":21,""Mall"":20,""Gas St...","{""Walmart"":54,""Mall"":53,""Gas Stations"":41,""Hos...","[110,114,124,133,164,167,225,236,230,239,263,3...","[324,345,348,362,356,478,574,739,683,817,824,8...","[334,378,362,398,425,483,582,691,833,869,881,9...","[449,470,423,465,494,593,664,783,797,775,715,8...","[423,477,462,474,503,589,837,964,1068,1007,112...","[485,512,505,516,619,712,896,1085,1245,1110,12...","[391,430,384,427,416,512,487,515,565,624,613,6...","[443,433,415,465,485,512,511,516,594,635,685,6...","{""android"":2740,""ios"":2349}",US,NE,2023,06
2,550939606003,Census Block Group,Census Block Group,2023-06-01 00:00:00.000,2023-07-01 00:00:00.000,"{""Monday"":4,""Tuesday"":4,""Wednesday"":4,""Thursda...",33042,4781,"[1594,1195,789,1047,930,927,726,928,1295,1130,...","[0,0,5,42,74,42,23,59,82,145,70,72,81,71,64,11...","{""550939606003"":305,""550939606001"":85,""2704908...","{"

In [20]:
import os
sqlite_fname = os.path.join(saved_dir, f'{year}_{month}.db')
sqlite_fname

'D:\\SafeGraph\\Neighborhood_Patterns_reorganized\\2023_06.db'

In [21]:
def create_CBG_index_table(sqlite_fname):
    conn = sqlite3.connect(sqlite_fname)
    curs = conn.cursor()
     
    # write the CBG index table
    # Convert the dataframe variable "CBG_df" into Sqlite database table "CBG_index", 
    # set the column "area" as the index column.
    CBG_df.set_index('area', inplace=True)
    CBG_df.to_sql('CBG_index', conn, if_exists='replace')
    
    # Close the connection
    conn.close()

# create_CBG_index_table(sqlite_fname)

In [22]:
# Create a table named "OD" if not exist. The columns are: origin, destination, stop. 
# The first two columns should be index. All datatypes are long int.

def create_OD_table(sqlite_fname):
    conn = sqlite3.connect(sqlite_fname)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS OD (
        origin INTEGER NOT NULL,
        destination INTEGER NOT NULL,
        stop INTEGER,
        PRIMARY KEY (origin, destination)
    );
    ''')
    
    conn.commit()
    conn.close()

# create_OD_table(sqlite_fname=sqlite_fname)

In [23]:
'''
update the "stop" coloumn in OD table by "origin" and "destination". 
If the record exists, adding the previous value. 
If not exists, insert a new record.
'''

conn = sqlite3.connect(sqlite_fname)


def update_stop_value(origin, destination, stop_increment, conn):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO OD(origin, destination, stop)
    VALUES (?, ?, ?)
    ON CONFLICT(origin, destination)
    DO UPDATE SET stop=stop + ?;
    ''', (origin, destination, stop_increment, stop_increment))



# origin = 1
# destination = 2
# stop_increment = 30
# update_stop_value(origin, destination, stop_increment, conn)

# conn.close()

In [24]:
def remove_all_rows_from_OD(db_path):
    """
    Remove all rows from the "OD" table in the SQLite database.
    
    Parameters:
    - db_path: path to the SQLite database file
    
    Returns:
    - None
    """
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # Execute the DELETE command
    cursor.execute("DELETE FROM OD")
    
    # Commit the transaction
    conn.commit()
    
    # Close the connection
    conn.close()

# Usage:
# db_path = sqlite_fname
# remove_all_rows_from_OD(sqlite_fname)

In [25]:
def create_neighborhood_table(sqlite_fname):
    conn = sqlite3.connect(sqlite_fname)
    curs = conn.cursor()
     
    columns = ['area', 'raw_stop_counts', 'raw_device_counts',  'median_dwell', 'stops_by_each_hour']
    
    np_table_df = np_df[columns]
    # print(np_table_df.columns)
    
    np_table_df.set_index('area', inplace=True)
    np_table_df.to_sql('neighborhood_patterns', conn, if_exists='replace')
    
    # Close the connection
    conn.commit()
    conn.close()

# create_neighborhood_table(sqlite_fname)

In [26]:
def split_home_area_stops():
    remove_all_rows_from_OD(sqlite_fname)

    conn = sqlite3.connect(sqlite_fname)
    for idx, row in tqdm(np_df.iloc[:].iterrows()):
        # stops_by_each_hour = row['stops_by_each_hour']
        # stops_by_each_hour = json.loads(stops_by_each_hour)
        # print(stops_by_each_hour)
        area = row['area']
        raw_device_counts = int(row['raw_device_counts'])
        raw_stop_counts = int(row['raw_stop_counts'])
        stop_per_device = raw_stop_counts / raw_device_counts
        # print(f'area: {area}', f"stop_per_device: {stop_per_device: 0.3f}", )
        
        device_home_areas = row['device_home_areas']
        device_home_areas = json.loads(device_home_areas)
        destination = row['area']
        origins = row['device_home_areas']
        # print(device_home_areas)
        # if origin in device_home_areas.keys():
            # home_device_ratio =  device_home_areas[area] / raw_device_counts
            # print(f"{area} home device count: ", device_home_areas[area], f"{home_device_ratio: 0.3f}")
            # print()
        for index, (origin, stops) in enumerate(device_home_areas.items()):
            try:
                origin_idx = CBG_dict[origin]
                destination_idx = CBG_dict[area]
                update_stop_value(origin=origin_idx, destination=destination_idx, stop_increment=stops, conn=conn)
            except:
                continue
        conn.commit()
    conn.close()

In [27]:
conn = sqlite3.connect(sqlite_fname)
for idx, row in tqdm(np_df.sample(10).iterrows()):
    stops_by_each_hour = row['stops_by_each_hour']
    stops_by_each_hour = json.loads(stops_by_each_hour)
    # print(stops_by_each_hour)
    area = row['area']
    raw_device_counts = int(row['raw_device_counts'])
    raw_stop_counts = int(row['raw_stop_counts'])
    stop_per_device = raw_stop_counts / raw_device_counts
    # print(f'area: {area}', f"stop_per_device: {stop_per_device: 0.3f}", )
    print(f'area: {area}', f"sum of stops_by_each_hour: {sum(stops_by_each_hour)}", )
    print(f'area: {area}', f"raw_stop_counts: {raw_stop_counts}, raw_device_counts: {raw_device_counts}, stop_per_device: {stop_per_device:.1f}", )
    
    device_home_areas = row['device_home_areas']
    device_home_areas = json.loads(device_home_areas)
    destination = row['area']
    origins = row['device_home_areas']
    # print(device_home_areas)
    # print(device_home_areas)
    # if origin in device_home_areas.keys():
        # home_device_ratio =  device_home_areas[area] / raw_device_counts
        # print(f"{area} home device count: ", device_home_areas[area], f"{home_device_ratio: 0.3f}")
        # print()
    for index, (origin, stops) in enumerate(device_home_areas.items()):
        try:
            origin_idx = CBG_dict[origin]
            destination_idx = CBG_dict[area]
            # update_stop_value(origin=origin_idx, destination=destination_idx, stop_increment=stops, conn=conn)
        except:
            continue
    conn.commit()
conn.close()

10it [00:00, 714.67it/s]

area: 260992264001 sum of stops_by_each_hour: 73452
area: 260992264001 raw_stop_counts: 73846, raw_device_counts: 13746, stop_per_device: 5.4
area: 180479698003 sum of stops_by_each_hour: 24942
area: 180479698003 raw_stop_counts: 25212, raw_device_counts: 2297, stop_per_device: 11.0
area: 171770005001 sum of stops_by_each_hour: 14107
area: 171770005001 raw_stop_counts: 14295, raw_device_counts: 2865, stop_per_device: 5.0
area: 360594101004 sum of stops_by_each_hour: 1357
area: 360594101004 raw_stop_counts: 1361, raw_device_counts: 867, stop_per_device: 1.6
area: 120350602142 sum of stops_by_each_hour: 26919
area: 120350602142 raw_stop_counts: 27324, raw_device_counts: 5160, stop_per_device: 5.3
area: 181410115052 sum of stops_by_each_hour: 88390
area: 181410115052 raw_stop_counts: 88668, raw_device_counts: 18467, stop_per_device: 4.8
area: 261635579001 sum of stops_by_each_hour: 9522
area: 261635579001 raw_stop_counts: 9692, raw_device_counts: 2607, stop_per_device: 3.7
area: 350130010

In [28]:
df.columns

Index(['area', 'area_type', 'origin_area_type', 'date_range_start',
       'date_range_end', 'day_counts', 'raw_stop_counts', 'raw_device_counts',
       'stops_by_day', 'stops_by_each_hour', 'device_home_areas',
       'weekday_device_home_areas', 'weekend_device_home_areas',
       'breakfast_device_home_areas', 'lunch_device_home_areas',
       'afternoon_tea_device_home_areas', 'dinner_device_home_areas',
       'nightlife_device_home_areas', 'work_hours_device_home_areas',
       'work_behavior_device_home_areas', 'device_daytime_areas',
       'distance_from_home', 'distance_from_primary_daytime_location',
       'median_dwell', 'top_same_day_brand', 'top_same_month_brand',
       'popularity_by_each_hour', 'popularity_by_hour_monday',
       'popularity_by_hour_tuesday', 'popularity_by_hour_wednesday',
       'popularity_by_hour_thursday', 'popularity_by_hour_friday',
       'popularity_by_hour_saturday', 'popularity_by_hour_sunday',
       'device_type', 'iso_country_code', 're

In [29]:
df[['area', 'raw_stop_counts', 'raw_device_counts',  'median_dwell', 'stops_by_each_hour']].memory_usage(deep=True).sum()  / (2 **30)

0.41149138286709785

In [40]:
df['STOPS_BY_DAY']

0        [208,162,223,224,92,215,261,174,260,218,326,15...
1        [1842,1829,1559,1210,1200,1150,1338,1712,1755,...
2        [1594,1195,789,1047,930,927,726,928,1295,1130,...
3        [280,354,256,461,475,677,451,321,401,479,187,2...
4        [313,430,408,450,351,886,451,358,432,808,613,2...
                               ...                        
20952    [2619,2600,2303,1380,3382,2773,2634,2639,2966,...
20953    [241,129,294,223,215,303,204,457,451,513,344,3...
20954    [958,1126,1618,804,807,995,980,1092,1460,705,5...
20955    [142,92,79,77,133,149,173,175,280,127,78,132,1...
20956    [64,144,188,154,112,162,168,199,141,171,69,143...
Name: STOPS_BY_DAY, Length: 220684, dtype: object

## Get ACS data

In [1]:
# ! pip install CensusData
import pandas as pd
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
pd.set_option('display.precision', 2)

In [53]:
ACS2019_fname = r'G:\covid_mobility_results\new_census_data\ACS_2019_5YR_BG\ACS_race_cbsa_2019.csv'
ACS_2019_df = pd.read_csv(ACS2019_fname)
ACS_2019_df['CBG_FIPS'] = ACS_2019_df['GEOID_Data'].str[-12:]
ACS_2019_df.columns
ACS_2019_df

,OBJECTID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,INTPTLAT,INTPTLON,Shape_Area,GEOID_Data,county_code,cbg_total_population,p_asian,p_black,p_white,CBSA Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,GEOID,total_household_income,total_households,mean_household_income,median_household_income,CBG_FIPS
0,1.0,1,73,5903,3,33.60,-86.68,2.03e+06,15000US010730059033,1073,1991,0.02,0.69,0.29,13820,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",Jefferson County,Alabama,1,73,15000US010730059033,3.90e+07,811.0,48069.79,41875,010730059033
1,2.0,1,73,5903,1,33.60,-86.67,4.29e+06,15000US010730059031,1073,2187,0.00,0.45,0.48,13820,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",Jefferson County,Alabama,1,73,15000US010730059031,4.97e+07,612.0,81200.48,92649,010730059031
2,3.0,1,73,5905,3,33.61,-86.69,2.42e+06,15000US010730059053,1073,2741,0.00,0.79,0.18,13820,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",Jefferson County,Alabama,1,73,15000US010730059053,5.54e+07,867.0,63877.91,48906,010730059053
3,4.0,1,73,5905,1,33.60,-86.70,1.44e+06,15000US010730059051,1073,1213,0.00,0.77,0.23,13820,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",Jefferson County,Alabama,1,73,15000US010730059051,2.63e+07,517.0,50961.84,40679,010730059051
4,5.0,1,73,5702,3,33.47,-86.88,7.35e+05,15000US010730057023,1073,412,0.00,0.96,0.04,13820,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",Jefferson County,Alabama,1,73,15000US010730057023,8.68e+06,205.0,42353.26,38669,010730057023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202620,220133.0,72,47,530600,1,18.26,-66.32,6.03e+06,15000US720475306001,72047,1386,0.00,0.10,0.76,41980,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,"San Juan-Bayamón, PR",Corozal Municipio,Puerto Rico,72,47,15000US720475306001,1.22e+07,419.0,29068.95,0,720475306001
202621,220166.0,72,47,530100,2,18.35,-66.33,3.52e+06,15000US720475301002,72047,905,0.00,0.10,0.85,41980,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,"San Juan-Bayamón, PR",Corozal Municipio,Puerto Rico,72,47,15000US720475301002,6.03e+06,333.0,18122.84,13302,720475301002
202622,220167.0,72,47,530300,1,18.34,-66.31,3.92e+05,15000US720475303001,72047,643,0.00,0.00,0.96,41980,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,"San Juan-Bayamón, PR",Corozal Municipio,Puerto Rico,72,47,15000US720475303001,5.75e+06,225.0,25555.32,0,720475303001
202623,220265.0,72,47,530300,2,18.34,-66.32,6.71e+05,15000US720475303002,72047,1893,0.00,0.16,0.79,41980,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,"San Juan-Bayamón, PR",Corozal Municipio,Puerto Rico,72,47,15000US720475303002,8.22e+06,497.0,16549.13,8235,720475303002


In [62]:
import sqlite3
import pandas as pd

def fetch_random_rows_to_dataframe(sqlite_fname):
    """
    Fetch random 10 rows from the table "neighborhood_patterns" in SQLite database,
    and return them as a Pandas DataFrame.

    Parameters:
    - sqlite_fname: path to the SQLite database file

    Returns:
    - DataFrame containing random 10 rows
    """
    # Connect to the database
    conn = sqlite3.connect(sqlite_fname)
    
    # SQL query to fetch random 10 rows
    sql_query = "SELECT * FROM neighborhood_patterns ORDER BY RANDOM() LIMIT 10;"
    
    # Use Pandas to execute the SQL query and fetch the result into a DataFrame
    df = pd.read_sql_query(sql_query, conn)

    # Close the connection
    conn.close()

    return df

# Usage:
# sqlite_fname = 'path_to_your_database_file.db'
random_rows_df = fetch_random_rows_to_dataframe(sqlite_fname)
# print(random_rows_df)

random_rows_df = random_rows_df.merge(ACS_2019_df, left_on='area', right_on='CBG_FIPS')
random_rows_df

,area,raw_stop_counts,raw_device_counts,median_dwell,stops_by_each_hour,OBJECTID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,INTPTLAT,INTPTLON,Shape_Area,GEOID_Data,county_code,cbg_total_population,p_asian,p_black,p_white,CBSA Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,GEOID,total_household_income,total_households,mean_household_income,median_household_income,CBG_FIPS
0,202090426003,15116,3454,14.13,"[12,0,0,14,18,7,7,8,32,49,19,0,16,27,38,100,34...",77416.0,20,209,42600,3,39.08,-94.63,9.28e+05,15000US202090426003,20209,752,5.32e-03,2.66e-03,0.84,28140,312.0,"Kansas City, MO-KS",Metropolitan Statistical Area,"Kansas City-Overland Park-Kansas City, MO-KS",Wyandotte County,Kansas,20,209,15000US202090426003,9.98e+06,218.0,45767.90,27333,202090426003
1,220170237003,13504,2643,19.08,"[13,0,6,0,16,0,57,44,16,8,73,16,21,42,27,71,47...",82916.0,22,17,23700,3,32.43,-93.77,2.02e+06,15000US220170237003,22017,2114,0.00e+00,9.70e-01,0.02,43340,508.0,"Shreveport-Bossier City, LA",Metropolitan Statistical Area,"Shreveport-Bossier City-Minden, LA",Caddo Parish,Louisiana,22,17,15000US220170237003,1.47e+07,599.0,24482.07,24306,220170237003
2,260750054001,10619,1992,18.98,"[0,0,0,0,0,39,30,35,8,37,22,11,42,0,49,7,12,10...",96033.0,26,75,5400,1,42.24,-84.51,1.32e+07,15000US260750054001,26075,1585,0.00e+00,0.00e+00,1.00,27100,NaN,"Jackson, MI",Metropolitan Statistical Area,NaN,Jackson County,Michigan,26,75,15000US260750054001,4.55e+07,478.0,95125.09,79167,260750054001
3,340130049002,3984,719,121.97,"[0,0,0,0,0,0,0,0,6,0,0,17,12,41,20,0,15,0,12,1...",121020.0,34,13,4900,2,40.71,-74.21,1.05e+05,15000US340130049002,34013,965,0.00e+00,1.00e+00,0.00,35620,408.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,"New York-Newark, NY-NJ-CT-PA",Essex County,New Jersey,34,13,15000US340130049002,1.92e+07,381.0,50366.97,34946,340130049002
4,270270301041,6409,995,69.27,"[0,0,0,0,0,0,0,0,0,0,0,18,0,8,6,22,31,26,0,0,9...",106038.0,27,27,30104,1,46.69,-96.67,2.39e+08,15000US270270301041,27027,622,6.43e-03,3.22e-03,0.96,22020,244.0,"Fargo, ND-MN",Metropolitan Statistical Area,"Fargo-Wahpeton, ND-MN",Clay County,Minnesota,27,27,15000US270270301041,2.15e+07,258.0,83303.81,78750,270270301041
5,360810865001,21416,5635,9.88,"[0,0,0,0,0,0,55,32,115,123,77,96,92,95,39,60,9...",128215.0,36,81,86500,1,40.76,-73.83,7.99e+04,15000US360810865001,36081,2001,6.91e-01,3.95e-02,0.22,35620,408.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,"New York-Newark, NY-NJ-CT-PA",Queens County,New York,36,81,15000US360810865001,5.52e+07,1096.0,50341.74,31250,360810865001
6,391535318013,11343,2548,42.70,"[0,0,0,0,0,25,11,37,31,15,21,17,16,11,18,12,28...",152163.0,39,153,531801,3,40.99,-81.54,1.87e+06,15000US391535318013,39153,1035,7.44e-02,1.84e-02,0.89,10420,184.0,"Akron, OH",Metropolitan Statistical Area,"Cleveland-Akron-Canton, OH",Summit County,Ohio,39,153,15000US391535318013,3.87e+07,519.0,74551.57,67266,391535318013
7,211110117091,25159,2984,48.63,"[10,16,0,9,15,11,102,48,94,43,48,48,19,32,94,1...",81508.0,21,111,11709,1,38.09,-85.61,1.54e+07,15000US211110117091,21111,2331,0.00e+00,4.80e-02,0.93,31140,350.0,"Louisville/Jefferson County, KY-IN",Metropolitan Statistical Area,Louisville/Jefferson County--Elizabethtown--Ba...,Jefferson County,Kentucky,21,111,15000US211110117091,7.21e+07,893.0,80685.42,73365,211110117091


In [100]:
def show_CBG_samples(sample_df):
    for idx, row in tqdm(sample_df.iterrows()):
        stops_by_each_hour = row['stops_by_each_hour']
        # stops_by_each_hour = json.loads(stops_by_each_hour)
        # print(stops_by_each_hour)
        area = row['area']
        raw_device_counts = int(row['raw_device_counts'])
        raw_stop_counts = int(row['raw_stop_counts'])
        stop_per_device = raw_stop_counts / raw_device_counts
        # print(f'area: {area}', f"stop_per_device: {stop_per_device: 0.3f}", )
        sampling_rate = row['raw_device_counts'] / row['cbg_total_population']
        print(f'area: {area}', f"sum of stops_by_each_hour: {sum(stops_by_each_hour)}, ",
              f"sampling_rate: {sampling_rate:0.3f}")
        print(f'area: {area}', f"raw_stop_counts: {raw_stop_counts}, ",
              f"raw_device_counts: {raw_device_counts},",
              f"stop_per_device: {stop_per_device:.1f}", 
               f"cbg_total_population:", row['cbg_total_population'])
        print()
        # device_home_areas = row['device_home_areas']
        # device_home_areas = json.loads(device_home_areas)
        # destination = row['area']
        # origins = row['device_home_areas']

random_rows_df = fetch_random_rows_to_dataframe(sqlite_fname)
random_rows_df['stops_by_each_hour'] = random_rows_df['stops_by_each_hour'].apply(json.loads)
random_rows_df = random_rows_df.merge(ACS_2019_df, left_on='area', right_on='CBG_FIPS')
show_CBG_samples(random_rows_df)

10it [00:00, 1342.74it/s]

area: 090091803001 sum of stops_by_each_hour: 13232,  sampling_rate: 4.056
area: 090091803001 raw_stop_counts: 13499,  raw_device_counts: 3062, stop_per_device: 4.4 cbg_total_population: 755

area: 421314002001 sum of stops_by_each_hour: 10597,  sampling_rate: 1.307
area: 421314002001 raw_stop_counts: 10645,  raw_device_counts: 1472, stop_per_device: 7.2 cbg_total_population: 1126

area: 121270906001 sum of stops_by_each_hour: 22765,  sampling_rate: 3.890
area: 121270906001 raw_stop_counts: 23095,  raw_device_counts: 4466, stop_per_device: 5.2 cbg_total_population: 1148

area: 211379201023 sum of stops_by_each_hour: 14218,  sampling_rate: 0.908
area: 211379201023 raw_stop_counts: 14390,  raw_device_counts: 1486, stop_per_device: 9.7 cbg_total_population: 1636

area: 220550021013 sum of stops_by_each_hour: 11783,  sampling_rate: 3.181
area: 220550021013 raw_stop_counts: 12019,  raw_device_counts: 3222, stop_per_device: 3.7 cbg_total_population: 1013

area: 132450015002 sum of stops_by_e

In [15]:
import sqlite3
import pandas as pd

def fetch_all_OD_to_dataframe(sqlite_fname):
   
    # Connect to the database
    conn = sqlite3.connect(sqlite_fname)
     
    sql_query = "SELECT * FROM OD limit 10"
    
    # Use Pandas to execute the SQL query and fetch the result into a DataFrame
    df = pd.read_sql_query(sql_query, conn)

    # Close the connection
    conn.close()

    return df

sqlite_fname = os.path.join(saved_dir, f'2023_06.db')
print(sqlite_fname)
conn = sqlite3.connect(sqlite_fname)
 
sql_query = "SELECT * FROM OD;"
# Close the connection

# Use Pandas to execute the SQL query and fetch the result into a DataFrame
OD_df = pd.read_sql_query(sql_query, conn)
cursor = conn.cursor()

# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the rows and extract table names
# tables = [table[0] for table in cursor.fetchall()]

conn.close()


OD_df
# tables


D:\SafeGraph\Neighborhood_Patterns_reorganized\2023_06.db


,origin,destination,stop
0,217823,217760,6
1,217757,217760,4
2,217821,217760,4
3,217828,217760,4
4,217823,217749,6
...,...,...,...
185534602,57944,58222,4
185534603,58146,58222,4
185534604,58098,58222,4
185534605,118432,58222,4


In [39]:
fetch_all_OD_to_dataframe(sqlite_fname)

In [48]:

# CBGs = censusdata.download('acs5', 2015,
#                              censusdata.censusgeo([('state', '17'), ('county', '031'), ('block group', '*')]),
#                              ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
#                               'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
#                               'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
#                               'B15003_014E', 'B15003_015E', 'B15003_016E'])

# CBGs

# 3d iterative proportional fitting

https://edyhsgr.github.io/IPFDescription/AKDOLWDIPFTHREED.pdf

https://u.demog.berkeley.edu/~eddieh/IPFDescription/AKDOLWDIPFTWOD.pdf

# Get the visitors of each CBG

In [49]:
import numpy as np
from netCDF4 import Dataset

# Create a sample matrix (for the sake of demonstration)
matrix = np.random.rand(220000, 220000 )
matrix

MemoryError: Unable to allocate 361. GiB for an array with shape (220000, 220000) and data type float64

In [ ]:


# Create a new netCDF file
rootgrp = Dataset("matrix.nc", "w", format="NETCDF4")

# Create dimensions for the matrix
dim_x = rootgrp.createDimension("x", 220684)
dim_y = rootgrp.createDimension("y", 220684)

# Create a variable in the file for the matrix and set its data
matrix_var = rootgrp.createVariable("matrix", "f8", ("x", "y"), zlib=True, complevel=4) # "f8" is for double precision float
matrix_var[:, :] = matrix

# Close the netCDF file
rootgrp.close()

In [31]:
df2 = pd.read_csv(r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\05\01\data_01af1467-0604-be6e-0043-0b8700e85436_13_7_0.csv.gz')
df2

,AREA,AREA_TYPE,ORIGIN_AREA_TYPE,DATE_RANGE_START,DATE_RANGE_END,DAY_COUNTS,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,STOPS_BY_DAY,STOPS_BY_EACH_HOUR,DEVICE_HOME_AREAS,WEEKDAY_DEVICE_HOME_AREAS,WEEKEND_DEVICE_HOME_AREAS,BREAKFAST_DEVICE_HOME_AREAS,LUNCH_DEVICE_HOME_AREAS,DINNER_DEVICE_HOME_AREAS,NIGHTLIFE_DEVICE_HOME_AREAS,WORK_HOURS_DEVICE_HOME_AREAS,WORK_BEHAVIOR_DEVICE_HOME_AREAS,DEVICE_DAYTIME_AREAS,DISTANCE_FROM_HOME,DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION,MEDIAN_DWELL,TOP_SAME_DAY_BRAND,TOP_SAME_MONTH_BRAND,POPULARITY_BY_EACH_HOUR,POPULARITY_BY_HOUR_MONDAY,POPULARITY_BY_HOUR_TUESDAY,POPULARITY_BY_HOUR_WEDNESDAY,POPULARITY_BY_HOUR_THURSDAY,POPULARITY_BY_HOUR_FRIDAY,POPULARITY_BY_HOUR_SATURDAY,POPULARITY_BY_HOUR_SUNDAY,DEVICE_TYPE,ISO_COUNTRY_CODE,REGION,Y,M
0,290470205003,Census Block Group,Census Block Group,2023-05-01 00:00:00.000,2023-06-01 00:00:00.000,"""{\""Monday\"":5,\""Tuesday\"":5,\""Wednesday\"":5,\...",14830,1766,"""[321,321,492,471,636,344,366,416,708,377,495,...","""[0,0,0,0,0,0,11,11,0,0,17,25,38,27,40,0,29,23...","""{\""290470205003\"":140,\""290470202013\"":12,\""2...","""{\""290470205003\"":113,\""290470202013\"":6,\""29...","""{\""290470205003\"":53,\""290470202013\"":4,\""291...","""{\""290470205003\"":44,\""291650303082\"":4,\""291...","""{\""290470205003\"":30,\""290470202013\"":4,\""291...","""{\""290470205003\"":51,\""290470202013\"":5,\""291...","""{\""290470205003\"":9,\""290470202013\"":5,\""2909...","""{\""290470205003\"":57,\""290470202013\"":5,\""291...","""{\""290470205003\"":18,\""290950166001\"":4,\""290...","""{\""290470205003\"":155,\""291650306001\"":9,\""20...",27,32,85.016667,"""{\""Mall\"":24,\""Walmart\"":17,\""Gas Stations\"":...","""{\""Mall\"":55,\""Walmart\"":43,\""Gas Stations\"":...","""[85,87,83,84,84,87,103,118,94,102,100,109,131...","""[514,509,541,547,594,613,668,680,541,571,597,...","""[643,651,640,623,697,722,763,719,682,653,651,...","""[576,638,653,612,692,706,781,808,751,634,676,...","""[384,398,414,425,467,506,525,552,416,441,494,...","""[480,480,479,495,569,625,671,710,605,554,555,...","""[412,424,430,420,424,442,414,453,411,428,382,...","""[463,512,492,543,519,524,580,566,596,607,572,...","""{\""android\"":1104,\""ios\"":423}""",US,MO,2023,5
1,291892122007,Census Block Group,Census Block Group,2023-05-01 00:00:00.000,2023-06-01 00:00:00.000,"""{\""Monday\"":5,\""Tuesday\"":5,\""Wednesday\"":5,\...",9277,2702,"""[237,329,255,196,352,237,350,116,382,345,227,...","""[0,0,0,0,0,0,0,0,12,0,0,15,11,26,13,49,8,38,2...","""{\""291892122007\"":47,\""291892108042\"":20,\""29...","""{\""291892122007\"":31,\""291892108042\"":14,\""29...","""{\""291892122007\"":11,\""291892121022\"":4,\""291...","""{\""291892122007\"":6,\""291892108042\"":4,\""2918...","""{\""291892122007\"":9,\""291892108042\"":4,\""2951...","""{\""291892122007\"":11,\""291892121022\"":4,\""291...","""{\""291892121014\"":4,\""291892122004\"":4,\""2918...","""{\""291892122007\"":17,\""291892108042\"":7,\""291...","""{\""291892122007\"":4,\""291892122001\"":4,\""2951...","""{\""291892122007\"":39,\""295101269003\"":26,\""29...",930,491,44.983333,"""{\""Mall\"":19,\""Universities and Colleges\"":9,...","""{\""Mall\"":67,\""Walmart\"":46,\""Gas Stations\"":...","""[49,53,47,49,54,55,53,57,54,41,65,78,82,68,48...","""[194,212,184,213,208,208,239,249,266,248,255,...","""[188,228,221,222,264,285,307,329,319,389,376,...","""[196,197,195,208,247,313,299,319,341,354,378,...","""[170,160,143,169,254,215,276,275,291,286,258,...","""[118,123,126,133,180,192,188,225,253,250,286,...","""[130,127,143,144,158,170,223,240,210,275,301,...","""[142,176,172,156,179,201,214,228,239,274,258,...","""{\""android\"":2181,\""ios\"":373}""",US,MO,2023,5
2,291892180033,Census Block Group,Census Block Group,2023-05-01 00:00:00.000,2023-06-01 00:00:00.000,"""{\""Monday\"":5,\""Tuesday\"":5,\""Wednesday\"":5,\...",24150,4625,"""[564,776,986,627,859,1030,896,394,966,812,895...","""[18,0,0,0,8,0,26,22,18,17,0,21,50,37,102,10,4...","""{\""291892180033\""

In [33]:
df2.columns

Index(['AREA', 'AREA_TYPE', 'ORIGIN_AREA_TYPE', 'DATE_RANGE_START',
       'DATE_RANGE_END', 'DAY_COUNTS', 'RAW_STOP_COUNTS', 'RAW_DEVICE_COUNTS',
       'STOPS_BY_DAY', 'STOPS_BY_EACH_HOUR', 'DEVICE_HOME_AREAS',
       'WEEKDAY_DEVICE_HOME_AREAS', 'WEEKEND_DEVICE_HOME_AREAS',
       'BREAKFAST_DEVICE_HOME_AREAS', 'LUNCH_DEVICE_HOME_AREAS',
       'DINNER_DEVICE_HOME_AREAS', 'NIGHTLIFE_DEVICE_HOME_AREAS',
       'WORK_HOURS_DEVICE_HOME_AREAS', 'WORK_BEHAVIOR_DEVICE_HOME_AREAS',
       'DEVICE_DAYTIME_AREAS', 'DISTANCE_FROM_HOME',
       'DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION', 'MEDIAN_DWELL',
       'TOP_SAME_DAY_BRAND', 'TOP_SAME_MONTH_BRAND', 'POPULARITY_BY_EACH_HOUR',
       'POPULARITY_BY_HOUR_MONDAY', 'POPULARITY_BY_HOUR_TUESDAY',
       'POPULARITY_BY_HOUR_WEDNESDAY', 'POPULARITY_BY_HOUR_THURSDAY',
       'POPULARITY_BY_HOUR_FRIDAY', 'POPULARITY_BY_HOUR_SATURDAY',
       'POPULARITY_BY_HOUR_SUNDAY', 'DEVICE_TYPE', 'ISO_COUNTRY_CODE',
       'REGION', 'Y', 'M'],
      dtype='obje

In [ ]:
['AREA', 'AREA_TYPE', 'ORIGIN_AREA_TYPE', 'DATE_RANGE_START',
       'DATE_RANGE_END', 'DAY_COUNTS', 'RAW_STOP_COUNTS', 'RAW_DEVICE_COUNTS',
       'STOPS_BY_DAY', 'STOPS_BY_EACH_HOUR', 'DEVICE_HOME_AREAS',
       'WEEKDAY_DEVICE_HOME_AREAS', 'WEEKEND_DEVICE_HOME_AREAS',
       'BREAKFAST_DEVICE_HOME_AREAS', 'LUNCH_DEVICE_HOME_AREAS',
       'DINNER_DEVICE_HOME_AREAS', 'NIGHTLIFE_DEVICE_HOME_AREAS',
       'WORK_HOURS_DEVICE_HOME_AREAS', 'WORK_BEHAVIOR_DEVICE_HOME_AREAS',
       'DEVICE_DAYTIME_AREAS', 'DISTANCE_FROM_HOME',
       'DISTANCE_FROM_PRIMARY_DAYTIME_LOCATION', 'MEDIAN_DWELL',
       'TOP_SAME_DAY_BRAND', 'TOP_SAME_MONTH_BRAND', 'POPULARITY_BY_EACH_HOUR',
       'POPULARITY_BY_HOUR_MONDAY', 'POPULARITY_BY_HOUR_TUESDAY',
       'POPULARITY_BY_HOUR_WEDNESDAY', 'POPULARITY_BY_HOUR_THURSDAY',
       'POPULARITY_BY_HOUR_FRIDAY', 'POPULARITY_BY_HOUR_SATURDAY',
       'POPULARITY_BY_HOUR_SUNDAY', 'DEVICE_TYPE', 'ISO_COUNTRY_CODE',
       'REGION', 'Y', 'M']

In [73]:
df['STOPS_BY_EACH_HOUR'] = df['STOPS_BY_EACH_HOUR'].apply(json.loads) 
df['STOPS_BY_EACH_HOUR'] = df['STOPS_BY_EACH_HOUR'].apply(json.loads) 

In [66]:
df['STOPS_BY_EACH_HOUR']

0        [0,0,0,0,0,0,0,0,0,20,0,0,0,0,0,0,30,5,0,0,14,...
1        [0,0,0,0,7,23,0,23,0,7,26,15,12,15,21,17,11,19...
2        [5,0,0,0,19,7,13,0,23,0,10,7,15,5,0,0,7,0,0,13...
3        [0,0,0,0,0,0,0,0,19,16,0,0,0,0,14,0,0,16,0,0,0...
4        [0,0,0,0,0,0,0,5,0,14,24,14,0,17,34,0,0,19,0,0...
                               ...                        
20243    [16,9,0,21,16,0,27,27,47,45,45,89,48,50,11,105...
20244    [46,25,44,0,46,27,71,66,92,54,87,59,137,125,43...
20245    [77,0,0,0,0,0,12,33,30,40,34,34,46,0,48,48,44,...
20246    [0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,13,0,7,0,14,0...
20247    [0,0,0,0,0,24,36,68,104,117,66,10,74,48,92,57,...
Name: STOPS_BY_EACH_HOUR, Length: 220692, dtype: object

In [80]:
# df.iloc[6]['STOPS_BY_EACH_HOUR']

In [78]:
len(df.iloc[6]['STOPS_BY_EACH_HOUR'])

744

In [38]:
# json.loads(df.iloc[0]['STOPS_BY_EACH_HOUR'])

In [68]:
len(json.loads())

1746